In [1]:
!pip install streamlit
!pip install joblib

   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB 1.3 MB/s eta 0:00:08
   ---------------------------------------- 0.1/10.1 MB 1.6 MB/s eta 0:00:07
   - -------------------------------------- 0.4/10.1 MB 3.2 MB/s eta 0:00:04
   --- ------------------------------------ 0.8/10.1 MB 5.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.1 MB 7.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.3/10.1 MB 9.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.5/10.1 MB 9.3 MB/s eta 0:00:01
   --------- ------------------------------ 2.5/10.1 MB 9.3 MB/s eta 0:00:01
   -------------- ------------------------- 3.8/10.1 MB 9.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.1/10.1 MB 11.5 MB/s eta 0:00:01
   ---------------------- ----------------- 5.6/10.1 MB 11.9 MB/s eta 0:00:01
   ---------------------- ----------------- 5.6/10.1 MB 11.9 MB/s eta 0:00:01
   

In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor

In [4]:
# Load the trained model and label encoder (ensure these are saved previously)
model = joblib.load("model_rf.pkl")
label_enc = joblib.load("label_encoder.pkl")
X_columns = joblib.load("X_columns.pkl")  # list of columns used during training

In [5]:
# Title
st.title("Gym Floor Pricing Simulator")
st.markdown("Aim: Recommend optimal $/sqft to achieve 45% GP or higher")

2025-07-09 16:16:37.582 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:37.772 
  command:

    streamlit run C:\Users\mipo\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-09 16:16:37.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:37.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:37.774 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:37.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:37.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [6]:
# User inputs
size_sqft = st.number_input("Job Size (sqft)", min_value=1000, max_value=30000, value=6000, step=100)
coats = st.selectbox("Number of Coats", [1, 2])
distance = st.number_input("Distance to Job (miles)", min_value=0, max_value=500, value=25)
labor_hours = st.number_input("Estimated Labor Hours", min_value=1.0, max_value=100.0, value=16.0, step=0.25)
am_name = st.selectbox("Account Manager", sorted(label_enc.classes_.tolist()))
concurrent_job = st.selectbox("Concurrent Job?", ["Yes", "No"])

2025-07-09 16:16:39.320 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:39.320 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:39.321 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:39.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:39.324 Session state does not function when running a script without `streamlit run`
2025-07-09 16:16:39.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:39.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:39.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16

In [7]:
# Convert inputs to model format
am_encoded = label_enc.transform([am_name])[0]
concurrent_flag = 1 if concurrent_job == "Yes" else 0

input_data = {
    'Size_sqft': size_sqft,
    'Coats': coats,
    'Distance': distance,
    'Labor_Hours': labor_hours,
    'AM': am_encoded,
    'Concurrent_Job': concurrent_flag
}

In [8]:
# Prediction function
def get_best_price_per_sqft(job_input, model, X_columns, target_gp=0.45):
    base_features = pd.DataFrame([job_input])
    sqft = job_input['Size_sqft']
    sqft_prices = np.arange(0.40, 1.20, 0.01)
    results = []

    for price_per_sqft in sqft_prices:
        total_quote = round(price_per_sqft * sqft, 2)
        test_row = base_features.copy()
        test_row['Quoted_Price'] = total_quote
        test_row = test_row.reindex(columns=X_columns, fill_value=0)
        predicted_gp = model.predict(test_row)[0]
        results.append((price_per_sqft, predicted_gp))

    for price_per_sqft, gp in results:
        if gp >= target_gp:
            return {
                'recommended_price_per_sqft': round(price_per_sqft, 2),
                'total_price': round(price_per_sqft * sqft, 2),
                'predicted_gp': round(gp, 4)
            }

    return {
        'recommended_price_per_sqft': None,
        'total_price': None,
        'predicted_gp': None
    }

In [9]:
# Run simulation
if st.button("Recommend Price"):
    result = get_best_price_per_sqft(input_data, model, X_columns)
    if result['recommended_price_per_sqft'] is not None:
        st.success(f"✅ Recommended Price: ${result['total_price']:,} ({result['recommended_price_per_sqft']}/sqft)")
        st.write(f"Predicted GP%: {result['predicted_gp']*100:.2f}%")
    else:
        st.error("❌ No quote in range meets 45% GP target. Consider increasing efficiency or base pricing.")

2025-07-09 16:16:45.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:45.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:45.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:45.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:45.523 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-09 16:16:45.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
